In [1]:
#Import statements
import pandas as pd
from bertopic import BERTopic
from bertopic.representation import ZeroShotClassification
from bertopic import BERTopic
from collections import defaultdict 

In [2]:
#Read submissions data
submissions=pd.read_json('cancer_submissions.zst', compression=dict(method='zstd'), \
                         lines=True)

In [3]:
submissions.head()

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_text,author_flair_text_color,brand_safe,can_gild,contest_mode,created_utc,...,treatment_tags,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,poll_data,gallery_data,is_gallery,call_to_action
0,1.0,[deleted],,None,None,dark,0.0,0.0,0.0,1225060992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,[deleted],,None,None,dark,0.0,0.0,0.0,1240211158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,cathyz,None,None,None,None,0.0,1.0,0.0,1246264550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,chris99460,None,None,None,None,0.0,1.0,0.0,1247023273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,ricemilk,None,None,None,None,0.0,1.0,0.0,1247845780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Read comments
comments=pd.read_json('cancer_comments.zst', compression=dict(method='zstd'), \
                         lines=True)

In [5]:
comments.head()

,retrieved_on,author,distinguished,downs,score,created_utc,body,subreddit,subreddit_id,ups,...,collapsed_because_crowd_control,author_premium,treatment_tags,top_awarded_type,comment_type,collapsed_reason_code,retrieved_utc,author_is_blocked,unrepliable_reason,editable
0,1.426047e+09,ronochu,None,0.0,0,1253101073,Good info about pancreas cancer,cancer,t5_2qixx,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.426063e+09,Mstuder,None,0.0,1,1254519758,This article is about a tea-related product so...,cancer,t5_2qixx,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.426063e+09,Mstuder,None,0.0,1,1254530520,This is a quick article about things you can d...,cancer,t5_2qixx,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.426068e+09,[deleted],None,0.0,1,1255070915,[deleted],cancer,t5_2qixx,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.426068e+09,[deleted],None,0.0,1,1255071543,[deleted],cancer,t5_2qixx,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
len(comments)

416127

In [7]:
#Combine text and filter submissions which have more than 3 comments
submissions_filtered=submissions[~submissions["selftext"].isin(["[deleted]","[removed]"])]
submissions_filtered=submissions_filtered[submissions["num_comments"] >=3]
submissions_filtered["combined_text"]=submissions_filtered["title"]+" "+submissions_filtered["selftext"]
submissions_filtered=submissions_filtered[["id","combined_text"]]

In [8]:
len(submissions_filtered)

28716

<h2> BERTopic </h2>

In [23]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(submissions_filtered.combined_text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [24]:
tm_info=topic_model.get_topic_info()

In [27]:
tm_info.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,13267,-1_her_she_to_and,"[her, she, to, and, the, my, of, in, is, that]",[Evening hospital thoughts. Very thankful for ...
1,0,701,0_him_dad_he_his,"[him, dad, he, his, father, hes, passed, my, w...","[Loss of my dad Hey, first of all, sorry for m..."
2,1,650,1_hair_head_wig_loss,"[hair, head, wig, loss, shave, bald, wigs, gro...","[Helping hair grow after chemo? Well, im final..."
3,2,568,2_lymphoma_hodgkins_neck_lymph,"[lymphoma, hodgkins, neck, lymph, nodes, node,...",[What does this mean...? Had some more swollen...
4,3,407,3_melanoma_mole_dermatologist_skin,"[melanoma, mole, dermatologist, skin, moles, b...",[i’m still angry at the people who told me it ...


<h2> ZERO shot classification with BERTopic </h2>

In [7]:
# Create your representation model
candidate_topics = ["managing pain", "treatment advice","support","rant","marijuana"]
representation_model = ZeroShotClassification(candidate_topics, model="facebook/bart-large-mnli")

In [8]:
# Use the representation model in BERTopic on top of the default pipeline
topic_model_zeroshot = BERTopic(representation_model=representation_model)

In [9]:
zero_shot_topic,zero_shot_prob=topic_model_zeroshot.fit_transform(submissions_filtered.combined_text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
topic_model_zeroshot.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12845,-1_to_and_the_her,"[to, and, the, her, she, my, of, in, is, was]",[My Moms Battles With Cancer My mom was first ...
1,0,857,0_him_he_dad_his,"[him, he, dad, his, hes, father, my, passed, w...","[Loss of my dad Hey, first of all, sorry for m..."
2,1,640,1_hair_head_wig_shave,"[hair, head, wig, shave, loss, bald, wigs, gro...","[Hair love , How to deal with hair loss? I sta..."
3,2,516,2_lymphoma_hodgkins_neck_lymph,"[lymphoma, hodgkins, neck, lymph, nodes, node,...",[Months of tests &amp; scans - biopsy for lymp...
4,3,380,3_insurance_pay_financial_cost,"[insurance, pay, financial, cost, coverage, bi...","[Finding an oncologist without insurance? Hi, ..."
...,...,...,...,...,...
253,252,10,252_internship_travel_quitting_travelling,"[internship, travel, quitting, travelling, par...","[Advice about travel Hey friends,\n\nI'm not t..."
254,253,10,253_melanoma_braftovimektovi_braftovi_mektovi,"[melanoma, braftovimektovi, braftovi, mektovi,...",[Difficulty with taf/mek side effects (stage 3...
255,254,10,254_treatment advice___,"[treatment advice, , , , , , , , , ]",[Melanoma questions &amp; treatment preparatio...
256,255,10,255_wife_breast_lfs_double,"[wife, breast, lfs, double, mastectomy, ductal...",[Sister was diagnosed with breast cancer last ...


In [11]:
cluster_dict=defaultdict(list)
for _,row in topic_model_zeroshot.get_topic_info().iterrows():
    for c in candidate_topics:
        if c in row["Representation"]:
            cluster_dict[c].append(row["Topic"])
            print(row["Representation"],row["Topic"])

['cannabis', 'marijuana', 'cbd', 'oil', 'thc', 'weed', 'medical', 'edibles', 'smoke', 'smoking'] 15
['support', '', '', '', '', '', '', '', '', ''] 16
['support', '', '', '', '', '', '', '', '', ''] 24
['support', 'groups', 'app', 'group', 'patients', 'online', 'caregivers', 'you', 'people', 'their'] 37
['support', '', '', '', '', '', '', '', '', ''] 38
['support', '', '', '', '', '', '', '', '', ''] 44
['support', '', '', '', '', '', '', '', '', ''] 62
['support', '', '', '', '', '', '', '', '', ''] 64
['support', '', '', '', '', '', '', '', '', ''] 66
['support', '', '', '', '', '', '', '', '', ''] 81
['treatment advice', '', '', '', '', '', '', '', '', ''] 88
['treatment advice', '', '', '', '', '', '', '', '', ''] 90
['support', '', '', '', '', '', '', '', '', ''] 96
['rant', '', '', '', '', '', '', '', '', ''] 105
['support', '', '', '', '', '', '', '', '', ''] 106
['treatment advice', '', '', '', '', '', '', '', '', ''] 112
['support', '', '', '', '', '', '', '', '', ''] 113
['su

In [45]:
cluster_dict

defaultdict(list,
            {'marijuana': [15, 215],
             'support': [16,
              24,
              37,
              38,
              44,
              62,
              64,
              66,
              81,
              96,
              106,
              113,
              117,
              146,
              150,
              152,
              158,
              189,
              220,
              233,
              249,
              250],
             'treatment advice': [88, 90, 112, 214, 254],
             'rant': [105, 135, 184, 256],
             'managing pain': [131, 174, 186]})

In [13]:
submissions_filtered["topic"]=zero_shot_topic

In [14]:
submissions_filtered.to_csv("final_zeroshot_bert_topic.csv",index=False)

In [15]:
submissions_filtered.head()

,id,combined_text,topic
28,a0d9k,Peter Criss Have Breast Cancer,-1
163,d76na,asbestos total ban is still not implemented......,-1
168,d9f3r,Starving cancer with food | Braincrave.com... ...,45
170,das76,Would anyone care to be a moderator in /r/canc...,74
177,dbpck,Worst case scenario:this mesothelioma vaccine ...,-1


In [ ]:
filterd_df=submissions_filtered[submissions_filtered["topic"].isin(cluster_dict["marijuana"])]
filterd_df["id"]="t3_"+filterd_df["id"]
filterd_df_comments=comments[comments["link_id"].isin(list(filterd_df.id))]
filterd_df_comments=filterd_df_comments[filterd_df_comments["link_id"]==filterd_df_comments["parent_id"]]
filterd_df_comments=filterd_df_comments[["link_id","parent_id","body"]]
filterd_df_comments=filterd_df_comments[~filterd_df_comments["body"].isin(["[deleted]","[removed]"])]
topic_model_comments = BERTopic()
topics_comments, probs_comments = topic_model_comments.fit_transform(filterd_df_comments.body)
filterd_df_comments["topic"]=topics_comments
filterd_df_comments.to_csv(t+"_comments_all_clusters.csv",index=False)

In [ ]:
filterd_df=submissions_filtered[submissions_filtered["topic"].isin(cluster_dict["support"])]
filterd_df["id"]="t3_"+filterd_df["id"]
filterd_df_comments=comments[comments["link_id"].isin(list(filterd_df.id))]
filterd_df_comments=filterd_df_comments[filterd_df_comments["link_id"]==filterd_df_comments["parent_id"]]
filterd_df_comments=filterd_df_comments[["link_id","parent_id","body"]]
filterd_df_comments=filterd_df_comments[~filterd_df_comments["body"].isin(["[deleted]","[removed]"])]
topic_model_comments = BERTopic()
topics_comments, probs_comments = topic_model_comments.fit_transform(filterd_df_comments.body)
filterd_df_comments["topic"]=topics_comments
filterd_df_comments.to_csv(t+"_comments_all_clusters.csv",index=False)

In [ ]:
filterd_df=submissions_filtered[submissions_filtered["topic"].isin(cluster_dict["treatment advice"])]
filterd_df["id"]="t3_"+filterd_df["id"]
filterd_df_comments=comments[comments["link_id"].isin(list(filterd_df.id))]
filterd_df_comments=filterd_df_comments[filterd_df_comments["link_id"]==filterd_df_comments["parent_id"]]
filterd_df_comments=filterd_df_comments[["link_id","parent_id","body"]]
filterd_df_comments=filterd_df_comments[~filterd_df_comments["body"].isin(["[deleted]","[removed]"])]
topic_model_comments = BERTopic()
topics_comments, probs_comments = topic_model_comments.fit_transform(filterd_df_comments.body)
filterd_df_comments["topic"]=topics_comments
filterd_df_comments.to_csv(t+"_comments_all_clusters.csv",index=False)

In [ ]:
filterd_df=submissions_filtered[submissions_filtered["topic"].isin(cluster_dict["rant"])]
filterd_df["id"]="t3_"+filterd_df["id"]
filterd_df_comments=comments[comments["link_id"].isin(list(filterd_df.id))]
filterd_df_comments=filterd_df_comments[filterd_df_comments["link_id"]==filterd_df_comments["parent_id"]]
filterd_df_comments=filterd_df_comments[["link_id","parent_id","body"]]
filterd_df_comments=filterd_df_comments[~filterd_df_comments["body"].isin(["[deleted]","[removed]"])]
topic_model_comments = BERTopic()
topics_comments, probs_comments = topic_model_comments.fit_transform(filterd_df_comments.body)
filterd_df_comments["topic"]=topics_comments
filterd_df_comments.to_csv(t+"_comments_all_clusters.csv",index=False)

In [ ]:
filterd_df=submissions_filtered[submissions_filtered["topic"].isin(cluster_dict["managing pain"])]
filterd_df["id"]="t3_"+filterd_df["id"]
filterd_df_comments=comments[comments["link_id"].isin(list(filterd_df.id))]
filterd_df_comments=filterd_df_comments[filterd_df_comments["link_id"]==filterd_df_comments["parent_id"]]
filterd_df_comments=filterd_df_comments[["link_id","parent_id","body"]]
filterd_df_comments=filterd_df_comments[~filterd_df_comments["body"].isin(["[deleted]","[removed]"])]
topic_model_comments = BERTopic()
topics_comments, probs_comments = topic_model_comments.fit_transform(filterd_df_comments.body)
filterd_df_comments["topic"]=topics_comments
filterd_df_comments.to_csv(t+"_comments_all_clusters.csv",index=False)